<a href="https://colab.research.google.com/github/opailo/TensorFlow_Advanced_Techniques_Specialization/blob/main/Course%201%3A%20Custom%20Models%2C%20Layers%2C%20and%20Loss%20Functions/Mini_ResNet_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import  Activation , Conv2D , BatchNormalization , Add , MaxPool2D , GlobalAveragePooling2D , Dense
import tensorflow_datasets as tfds
from tensorflow.python.keras.utils.vis_utils import plot_model

##Create Identity Block Class 

This code will define the identity block from the mini ResNet model seen in lecture 

In [ ]:
class IdentityBlock(Model):

  def __init__(self, filters, kernal_size):
    super(IdentityBlock, self).__init__(name = '')
    self.conv = Conv2D(filters, kernal_size, padding='same')
    self.norm = BatchNormalization()
    self.act = Activation('relu')
    self.add = Add()
  

  def call(self, input):
    x = self.conv(input)
    x = self.norm(x)
    x = self.act(x)

    x = self.conv(x)
    x = self.norm(x)

    x = self.add([x, input])
    x = self.act(x)

    return x

##Build the `Resnet` Class

This class will implement the code above as the `IdentityBlocks` of the `Resnet`

In [ ]:
class Resnet(Model):
  def __init__(self, num_classes, activation='softmax'):
    
    super(Resnet, self).__init__(name='')

    #Convolutional Layer with 7x7 Filter
    self.conv7 = Conv2D(64, 7, padding='same')
    
    #Batch Normalization Layer 
    self.norm = BatchNormalization()

    #3x3 Max Pooling Layer 
    self.pool = MaxPool2D((3,3))

    #Identity Block Layer 1
    self.idbl1 = IdentityBlock(64, 3)

    #Identity Block Layer 2
    self.idbl2 = IdentityBlock(64, 3)

    #Global pooling layer
    self.gpool = GlobalAveragePooling2D()

    #Make model flexible and capable of defining classification 
    #part later with however many classes that's required
    self.classifier = Dense(num_classes, 
                            activation = tf.keras.activations.get(activation))
    
  
  def call(self, input):
    x = self.conv7(input)
    x = self.norm(x)
    x = self.pool(x)
    x = self.idbl1(x)
    x = self.idbl2(x)
    x = self.gpool(x)
    x = self.classifier(x)

    return x

In [ ]:
resnet_model = Resnet(10)
resnet_model.compile(optimizer='adam' , loss = tf.keras.losses.sparse_categorical_crossentropy , metrics=['acc'])


In [ ]:
def preprocess(f):
    return tf.cast(f['image'] , 'float32') / 255.0  , f["label"]

dataset = tfds.load(name='mnist' , split=tfds.Split.TRAIN)
dataset = dataset.map(preprocess).batch(32)

local data directory. If you'd instead prefer to read directly from our public
GCS bucket (recommended if you're running on GCP), you can instead pass
`try_gcs=True` to `tfds.load` or set `data_dir=gs://tfds-data/datasets`.



Dl Completed...:   0%|          | 0/4 [00:00<?, ? file/s]


Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [ ]:
history = resnet_model.fit(dataset, epochs=3)

Epoch 1/3
1875/1875 [==============================] - 239s 126ms/step - loss: 0.1492 - acc: 0.9594
Epoch 2/3
1875/1875 [==============================] - 223s 119ms/step - loss: 0.0551 - acc: 0.9836
Epoch 3/3
1875/1875 [==============================] - 224s 120ms/step - loss: 0.0405 - acc: 0.9881


In [ ]:
dataset_test = tfds.load(name='mnist', split=tfds.Split.TEST)
dataset_test = dataset_test.map(preprocess).batch(32)

In [ ]:
resnet_model.evaluate(dataset_test)

In [ ]:
resnet_model.summary()

Model: ""
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  3200      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
  (IdentityBlock)            multiple                  37184     
                                                                 
  (IdentityBlock)            multiple                  37184     
                                                                 
 global_average_pooling2d (G  multiple                 0         


In [ ]:
plot_model(resnet_model, to_fil='resnet.png', show_shapes=True, show_layer_names=True)